In [1]:
#%pip install -r requirements.txt

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=400.0)

In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("data").load_data()


# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding()



c:\automatization\conf7nov\ollama\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(documents)

[Document(id_='61e80d07-0599-485f-852d-b2baca5bd9bb', embedding=None, metadata={'page_label': '3', 'file_name': 'Lowther.pdf', 'file_path': 'c:\\automatization\\conf7nov\\ollama\\data\\Lowther.pdf', 'file_type': 'application/pdf', 'file_size': 259947, 'creation_date': '2024-10-28', 'last_modified_date': '2024-10-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='AIR & SPACE OPERATIONS REVIEW   3\nUNMANNED  AIRCRAFT  SYSTEMSCombat Drones in Ukraine\nAdAm Lowther\nmAhbube  K. S iddiKi\nDrones are playing an important role in the war in Ukraine. Without a large conventional air \nforce, the Ukrainian military is employing a number of high- and low- end imported and domes -\ntically produced drones to devastating effect against Russian forces. This articl

In [6]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [7]:
from IPython.display import Markdown, display


# save to disk

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



In [8]:

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [11]:

# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

# Query Data from the persisted index
query_engine = index.as_query_engine(llm=llm)


In [19]:
prompt = 'What is the bayraktar?'

response = query_engine.query(prompt)
display(Markdown(f"<b>{response}</b>"))

ValueError: Expected where to have exactly one operator, got {} in query.